Based on TFX Keras Component Tuturial, altered for Sepsis prediction. https://www.tensorflow.org/tfx/tutorials/tfx/components_keras#setup

# Background

This notebook demonstrates how to use TFX in Jupyter on Red Hat OpenShift using Open Data Hub.

The ineractive notebook introduces slights differences in an actual pipeline, like orchestration and metadata artifacts.

# Orchestration

In production, you use an orchestrator (Apache Airflow, Kubeflow Pipelines, Argo, etc.) to orchestrate pre-defined pipeline graph of TFX components.

In a notebook, the notebook is the orchestrator.

# Metadata

In production, you access metadata through an API, like TFX ML Metadata "MLMD" API.

In production, MLMD stores metadata in a database like MySQL or SQLite.

In a notebook, payloads are stored in an ephemeral SQLite database under /tmp on the Jupyter server.

# Setup

In [144]:
# quietly upgrade pip
!pip install --upgrade pip -q

# install tfx as user
!pip install -Uq tfx

In [145]:
# import packages
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [146]:
# check versions installed
# expected TensorFlow version: 2.7.1 TFX version: 1.6.1
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.7.1
TFX version: 1.6.1


# Set up the pipeline paths

In [147]:
# check the relative path to the data
!ls -l ../../data/dataSepsis/csv_format

total 6516
-rw-rw-r--. 1 1000640000 1000640000    3032 Mar 10 17:58 attribute_definitions.csv
-rw-rw-r--. 1 1000640000 1000640000 1112373 Mar 10 17:58 pat_demog_labeled-dataSepsis.csv
-rw-rw-r--. 1 1000640000 1000640000 4126250 Mar 10 17:58 pat_labs_labeled-dataSepsis.csv
-rw-rw-r--. 1 1000640000 1000640000 1422189 Mar 10 17:58 pat_vitals_labeled-dataSepsis.csv


In [148]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Sepsis Pipeline example.
_taxi_root = os.path.join(_tfx_root, '../pipeline/sepsis_vitals')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), '../models/sepsis_vitals')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

In [149]:
# defines the temporary directory for the sepsis vitals data
_data_root = tempfile.mkdtemp(prefix='sepsis-data')
# defines the github path where the data exists, even though it is also cloned in
DATA_PATH = 'https://raw.githubusercontent.com/redhat-naps-da/mlops-prototype/main/data/dataSepsis/csv_format/pat_vitals_labeled-dataSepsis.csv'
# defines the full local temporary path
_data_filepath = os.path.join(_data_root, "pat_vitals_labeled-dataSepsis.csv")
# pulls the data into the newly created temporary filepath
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/sepsis-datajje_mizp/pat_vitals_labeled-dataSepsis.csv',
 <http.client.HTTPMessage at 0x7f22dc4f1b50>)

In [150]:
# view the first 10 lines of the data
!head {_data_filepath}

patient_id,record_date,record_time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,isSepsis
1,,,63,90,40.3,NaN,NaN,NaN,17,NaN,0
2,,,79,95,39.2,143,77,47,13,NaN,0
3,,,87,94,40.3,133,74,48,20,NaN,0
4,,,71,100,42.1,NaN,NaN,NaN,15,NaN,0
5,,,68,94.5,39.7,147.5,102,NaN,20,NaN,0
6,,,78,99,39.6,100,67,49.5,18,NaN,0
7,,,242,NaN,39.30,NaN,NaN,NaN,33,NaN,1
8,,,81,100,40.3,112,79.5,63,18,NaN,0
9,,,178,100,39.22,141,85,57,22,NaN,1


# Create the interactive context

In [151]:
context = InteractiveContext()

# TFX Components

## Data Ingestion with ExampleGen

In [152]:
# ExampleGen 
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
# enable cache
context.run(example_gen, enable_cache=False)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data /tmp/sepsis-datajje_mizp/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:1422189,xor_checksum:1647369591,sum_checksum:1647369591"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.1"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [153]:
# Examine ExampleGen artifact
# Expect to see train and eval
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/CsvExampleGen/examples/1


In [154]:
# Examine train examples
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "DBP"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "EtCO2"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "HR"
    value {
      int64_list {
        value: 63
      }
    }
  }
  feature {
    key: "MAP"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "O2Sat"
    value {
      float_list {
        value: 90.0
      }
    }
  }
  feature {
    key: "Resp"
    value {
      int64_list {
        value: 17
      }
    }
  }
  feature {
    key: "SBP"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "Temp"
    value {
      float_list {
        value: 40.29999923706055
      }
    }
  }
  feature {
    key: "isSepsis"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "patient_id"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
 

## Data Analysis with StatisticsGen

In [155]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=False)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "/tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.1"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        base_type: STATISTICS
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [156]:
# Visualize the output
context.show(statistics_gen.outputs['statistics'])

## Data Statistics with SchemaGen

In [157]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=False)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "/tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.1"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [158]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'DBP',FLOAT,required,single,-
'EtCO2',FLOAT,required,single,-
'HR',INT,required,single,-
'MAP',FLOAT,required,single,-
'O2Sat',FLOAT,required,single,-
'Resp',INT,required,single,-
'SBP',FLOAT,required,single,-
'Temp',FLOAT,required,single,-
'isSepsis',INT,required,single,-


## Detect anomalies with ExampleValidator

In [159]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator, enable_cache=False)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/ExampleValidator/anomalies/4/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/ExampleValidator/anomalies/4/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "/tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.1"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [160]:
# visualize the anomalies in a table
context.show(example_validator.outputs['anomalies'])

## Transform

In [161]:
_taxi_constants_module_file = 'taxi_constants.py'

In [162]:
%%writefile {_taxi_constants_module_file}

NUMERICAL_FEATURES = ['HR','O2Sat','Temp','SBP','MAP','DBP','Resp','EtCO2']

BUCKET_FEATURES = [
    #'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
    #'dropoff_longitude'
]
# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10

CATEGORICAL_NUMERICAL_FEATURES = [
    #'trip_start_hour', 'trip_start_day', 'trip_start_month',
    #'pickup_census_tract', 'dropoff_census_tract', 'pickup_community_area',
    #'dropoff_community_area'
]

CATEGORICAL_STRING_FEATURES = [
    #'payment_type',
    #'company',
]

# Number of vocabulary terms used for encoding categorical features.
VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized categorical are hashed.
OOV_SIZE = 10

# Keys
LABEL_KEY = 'isSepsis'
#FARE_KEY = 'fare'

def t_name(key):
  """
  Rename the feature keys so that they don't clash with the raw keys when
  running the Evaluator component.
  Args:
    key: The original feature key
  Returns:
    key with '_xf' appended
  """
  return key + '_xf'

Writing taxi_constants.py


In [163]:
_taxi_transform_module_file = 'taxi_transform.py'

In [164]:
%%writefile {_taxi_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import taxi_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(taxi_constants)

_NUMERICAL_FEATURES = taxi_constants.NUMERICAL_FEATURES
_BUCKET_FEATURES = taxi_constants.BUCKET_FEATURES
_FEATURE_BUCKET_COUNT = taxi_constants.FEATURE_BUCKET_COUNT
_CATEGORICAL_NUMERICAL_FEATURES = taxi_constants.CATEGORICAL_NUMERICAL_FEATURES
_CATEGORICAL_STRING_FEATURES = taxi_constants.CATEGORICAL_STRING_FEATURES
_VOCAB_SIZE = taxi_constants.VOCAB_SIZE
_OOV_SIZE = taxi_constants.OOV_SIZE
#_FARE_KEY = taxi_constants.FARE_KEY
_LABEL_KEY = taxi_constants.LABEL_KEY


def _make_one_hot(x, key):
  """Make a one-hot tensor to encode categorical features.
  Args:
    X: A dense tensor
    key: A string key for the feature in the input
  Returns:
    A dense one-hot tensor as a float list
  """
  integerized = tft.compute_and_apply_vocabulary(x,
          top_k=_VOCAB_SIZE,
          num_oov_buckets=_OOV_SIZE,
          vocab_filename=key, name=key)
  depth = (
      tft.experimental.get_vocabulary_size_by_name(key) + _OOV_SIZE)
  one_hot_encoded = tf.one_hot(
      integerized,
      depth=tf.cast(depth, tf.int32),
      on_value=1.0,
      off_value=0.0)
  return tf.reshape(one_hot_encoded, [-1, depth])


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  if not isinstance(x, tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _NUMERICAL_FEATURES:
    # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
    outputs[taxi_constants.t_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]), name=key)

  for key in _BUCKET_FEATURES:
    outputs[taxi_constants.t_name(key)] = tf.cast(tft.bucketize(
            _fill_in_missing(inputs[key]), _FEATURE_BUCKET_COUNT, name=key),
            dtype=tf.float32)

  for key in _CATEGORICAL_STRING_FEATURES:
    outputs[taxi_constants.t_name(key)] = _make_one_hot(_fill_in_missing(inputs[key]), key)

  for key in _CATEGORICAL_NUMERICAL_FEATURES:
    outputs[taxi_constants.t_name(key)] = _make_one_hot(tf.strings.strip(
        tf.strings.as_string(_fill_in_missing(inputs[key]))), key)

  # Was this passenger a big tipper?
  # taxi_fare = _fill_in_missing(inputs[_FARE_KEY])
  # tips = _fill_in_missing(inputs[_LABEL_KEY])
  # outputs[_LABEL_KEY] = tf.where(
  #    tf.math.is_nan(taxi_fare),
  #    tf.cast(tf.zeros_like(taxi_fare), tf.int64),
  #    # Test if the tip was > 20% of the fare.
  #    tf.cast(
  #        tf.greater(tips, tf.multiply(taxi_fare, tf.constant(0.2))), tf.int64))

  return outputs

Writing taxi_transform.py


In [165]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_taxi_transform_module_file))
context.run(transform, enable_cache=False)

INFO:absl:Generating ephemeral wheel package for '/opt/app-root/src/mlops-prototype/sepsis/notebooks/taxi_transform.py' (including modules: ['taxi_transform', 'taxi_constants']).
INFO:absl:User module package has hash fingerprint version 3de6afd2dfc393f411d4bff1fb85d38c4f2727949195ec649ee8afd818f476f1.
INFO:absl:Executing: ['/opt/app-root/bin/python3.8', '/tmp/tmp2ye90p6m/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpoie_artw', '--dist-dir', '/tmp/tmp4ab3sb7_']
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/_wheels/tfx_user_code_Transform-0.0+3de6afd2dfc393f411d4bff1fb85d38c4f2727949195ec649ee8afd818f476f1-py3-none-any.whl'; target user module is 'taxi_transform'.
INFO:absl:Full user module path is 'taxi_transform@/tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/_wheels/tfx_user_code_Transform-0.0+3de6afd2dfc393f411d4bff1fb85d38c4f2727949195ec649ee8afd818f476f1-py3-none-any.whl'
INFO:absl

INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/Transform/transform_graph/5/.temp_path/tftransform_tmp/f049e8acf01f44e7b7cc33be0ccb2db2/assets
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/Transform/transform_graph/5/.temp_path/tftransform_tmp/3147dc2bff754506a0e0a6fe8514f8ef/assets
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:struct2tensor is not available.


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 22
        uri: "/tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.1"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "/tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.1"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 23
        uri: "/tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.1"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 8
        type_id: 18
        uri: "/tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/Transform/pre_transform_schema/5"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
           

In [166]:
transform.outputs

{'transform_graph': Channel(
     type_name: TransformGraph
     artifacts: [Artifact(artifact: id: 5
 type_id: 22
 uri: "/tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/Transform/transform_graph/5"
 custom_properties {
   key: "name"
   value {
     string_value: "transform_graph"
   }
 }
 custom_properties {
   key: "producer_component"
   value {
     string_value: "Transform"
   }
 }
 custom_properties {
   key: "state"
   value {
     string_value: "published"
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.6.1"
   }
 }
 state: LIVE
 , artifact_type: id: 22
 name: "TransformGraph"
 )]
     additional_properties: {}
     additional_custom_properties: {}
 ),
 'transformed_examples': Channel(
     type_name: Examples
     artifacts: [Artifact(artifact: id: 6
 type_id: 14
 uri: "/tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/Transform/transformed_examples/5"
 properties {
   key: "split_names"
   value {
     string_value: "[\"tra

In [167]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transformed_metadata', 'transform_fn', 'metadata']

In [168]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "DBP_xf"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "EtCO2_xf"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "HR_xf"
    value {
      float_list {
        value: -0.6613534688949585
      }
    }
  }
  feature {
    key: "MAP_xf"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "O2Sat_xf"
    value {
      float_list {
        value: -2.6855642795562744
      }
    }
  }
  feature {
    key: "Resp_xf"
    value {
      float_list {
        value: -0.1939026564359665
      }
    }
  }
  feature {
    key: "SBP_xf"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "Temp_xf"
    value {
      float_list {
        value: 2.2171924114227295
      }
    }
  }
}

features {
  feature {
    key: "DBP_xf"
    value {
      float_list {
        value: -1.1920139789581299
      }
    }
  }
  feature

In [169]:
_taxi_trainer_module_file = 'taxi_trainer.py'

In [170]:
%%writefile {_taxi_trainer_module_file}

from typing import Dict, List, Text

import os
import glob
from absl import logging

import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_transform import TFTransformOutput

# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import taxi_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(taxi_constants)

_LABEL_KEY = taxi_constants.LABEL_KEY

_BATCH_SIZE = 40

print(_LABEL_KEY)


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY
      ),
      tf_transform_output.transformed_metadata.schema)

def _get_tf_examples_serving_signature(model, tf_transform_output):
  """Returns a serving signature that accepts `tensorflow.Example`."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_inference = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_example):
    """Returns the output to be used in the serving signature."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    # Remove label feature since these will not be present at serving time.
    raw_feature_spec.pop(_LABEL_KEY)
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_inference(raw_features)
    logging.info('serve_transformed_features = %s', transformed_features)

    outputs = model(transformed_features)
    # TODO(b/154085620): Convert the predicted labels from the model using a
    # reverse-lookup (opposite of transform.py).
    return {'outputs': outputs}

  return serve_tf_examples_fn


def _get_transform_features_signature(model, tf_transform_output):
  """Returns a serving signature that applies tf.Transform to features."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_eval = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def transform_features_fn(serialized_tf_example):
    """Returns the transformed_features to be fed as input to evaluator."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_eval(raw_features)
    logging.info('eval_transformed_features = %s', transformed_features)
    return transformed_features

  return transform_features_fn


def export_serving_model(tf_transform_output, model, output_dir):
  """Exports a keras model for serving.
  Args:
    tf_transform_output: Wrapper around output of tf.Transform.
    model: A keras model to export for serving.
    output_dir: A directory where the model will be exported to.
  """
  # The layer has to be saved to the model for keras tracking purpases.
  model.tft_layer = tf_transform_output.transform_features_layer()

  signatures = {
      'serving_default':
          _get_tf_examples_serving_signature(model, tf_transform_output),
      'transform_features':
          _get_transform_features_signature(model, tf_transform_output),
  }

  model.save(output_dir, save_format='tf', signatures=signatures)


def _build_keras_model(tf_transform_output: TFTransformOutput
                       ) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying taxi data.

  Args:
    tf_transform_output: [TFTransformOutput], the outputs from Transform

  Returns:
    A keras Model.
  """
  feature_spec = tf_transform_output.transformed_feature_spec().copy()
  feature_spec.pop(_LABEL_KEY)

  inputs = {}
  for key, spec in feature_spec.items():
    if isinstance(spec, tf.io.VarLenFeature):
      inputs[key] = tf.keras.layers.Input(
          shape=[None], name=key, dtype=spec.dtype, sparse=True)
    elif isinstance(spec, tf.io.FixedLenFeature):
      # TODO(b/208879020): Move into schema such that spec.shape is [1] and not
      # [] for scalars.
      inputs[key] = tf.keras.layers.Input(
          shape=spec.shape or [1], name=key, dtype=spec.dtype)
    else:
      raise ValueError('Spec type is not supported: ', key, spec)

  output = tf.keras.layers.Concatenate()(tf.nest.flatten(inputs))
  output = tf.keras.layers.Dense(100, activation='relu')(output)
  output = tf.keras.layers.Dense(70, activation='relu')(output)
  output = tf.keras.layers.Dense(50, activation='relu')(output)
  output = tf.keras.layers.Dense(20, activation='relu')(output)
  output = tf.keras.layers.Dense(1)(output)
  return tf.keras.Model(inputs=inputs, outputs=output)


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, _BATCH_SIZE)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, _BATCH_SIZE)

  model = _build_keras_model(tf_transform_output)

  model.compile(
      loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
      metrics=[tf.keras.metrics.BinaryAccuracy()])

  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

  # Export the model.
  export_serving_model(tf_transform_output, model, fn_args.serving_model_dir)

Writing taxi_trainer.py


In [172]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_taxi_trainer_module_file),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=10000),
    eval_args=tfx.proto.EvalArgs(num_steps=5000))
context.run(trainer, enable_cache=False)

INFO:absl:Generating ephemeral wheel package for '/opt/app-root/src/mlops-prototype/sepsis/notebooks/taxi_trainer.py' (including modules: ['taxi_transform', 'taxi_constants', 'taxi_trainer']).
INFO:absl:User module package has hash fingerprint version c10c4c609f930cd2308ff7e718009748f0c1f077a3b1ea15f3ba7f55fee59fd6.
INFO:absl:Executing: ['/opt/app-root/bin/python3.8', '/tmp/tmpx4qek0vj/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpge3u7ydq', '--dist-dir', '/tmp/tmp2zz88veu']
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/_wheels/tfx_user_code_Trainer-0.0+c10c4c609f930cd2308ff7e718009748f0c1f077a3b1ea15f3ba7f55fee59fd6-py3-none-any.whl'; target user module is 'taxi_trainer'.
INFO:absl:Full user module path is 'taxi_trainer@/tmp/tfx-interactive-2022-03-15T18_39_52.125170-wlu5q6xq/_wheels/tfx_user_code_Trainer-0.0+c10c4c609f930cd2308ff7e718009748f0c1f077a3b1ea15f3ba7f55fee59fd6-py3-none-any.whl'
INF

tips


ValueError: The `label_key` provided (tips) must be one of the following tensorsnames: dict_keys(['DBP_xf', 'EtCO2_xf', 'HR_xf', 'MAP_xf', 'O2Sat_xf', 'Resp_xf', 'SBP_xf', 'Temp_xf']).